# [O/H] Gas Profiles

In [1]:
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import json
from scipy.stats import binned_statistic
from scipy.stats import linregress
from scipy.optimize import curve_fit
from itertools import product

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/setuptools/_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.


In [2]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings
from auriga.parser import parse
from auriga.physics import Physics
from auriga.mathematics import linear

In [3]:
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(
    color=list(mcolors.TABLEAU_COLORS.values()))

In [37]:
figure_setup()
settings = Settings()

In [5]:
RXY_RANGE: tuple = (0, 30)
N_BINS: int = 30

In [21]:
def read_snapshot_data(simulation: str, of: str, to: str) -> tuple:
    """
    This method returns data of interest for this analysis.

    Parameters
    ----------
    simulation : str
        The simulation to consider.
    of : str
        The metal of which to calculate the metallicity.
    to : str
        The metal to which to calculate the metallicity.

    Returns
    -------
    pd.DataFrame
        A data frame with the properties.
    """

    settings = Settings()
    physics = Physics()

    s = Snapshot(simulation=simulation, loadonlytype=[0])
    s.add_extra_coordinates()
    s.add_metal_abundance(of=of, to=to)
    s.calculate_gas_temperature()
    s.calculate_hydrogen_number_density()

    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    df = pd.DataFrame()
    df[f"[{of}/{to}]_dex"] = s.metal_abundance[f"{of}/{to}"][is_main_obj]
    df["CylindricalRadius_ckpc"] = s.rho[is_main_obj]
    df["IsStarForming"] = \
        s.h_number_density[is_main_obj] > physics.star_forming_density
    df["IsCold"] = s.temperature[is_main_obj] < physics.critical_temperature

    # Remove invalid data
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)

    return df

In [29]:
def read_sample_data(galaxies: list) -> pd.DataFrame:
    df = pd.DataFrame()
    for galaxy in galaxies:
        gdf = read_snapshot_data(simulation=galaxy, of="O", to='H')
        mean, bin_edges, _ = binned_statistic(
                x=gdf["CylindricalRadius_ckpc"],
                values=gdf["[O/H]_dex"],
                statistic="mean", bins=N_BINS, range=RXY_RANGE)
        df[f"{galaxy.upper()}_<[O/H]>_dex"] = mean
    bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2
    df["BinCenters_ckpc"] = bin_centers
    return df

## [O/H] Profiles

In [39]:
sample = [f"au{i}_or_l4_s127" for i in settings.galaxies]
df = read_sample_data(galaxies=sample)

In [40]:
df["SampleAverage_<[O/H]>_dex"] = df[
    [f"{galaxy.upper()}_<[O/H]>_dex" for galaxy in sample]].mean(axis=1)
df["SampleDispersion_<[O/H]>_dex"] = df[
    [f"{galaxy.upper()}_<[O/H]>_dex" for galaxy in sample]].std(axis=1)

In [50]:
fig = plt.figure(figsize=(7.2, 7.2))
gs = fig.add_gridspec(nrows=6, ncols=5, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flatten():
    ax.set_xlim(RXY_RANGE)
    ax.set_ylim(-0.5, 1.5),
    ax.set_xticks([0, 5, 10, 15, 20, 25])
    ax.set_yticks([0, 0.5, 1.0])
    ax.set_xlabel(r"$r_{xy}$ [ckpc]")
    ax.set_ylabel(r"$\left< \mathrm{[O/H]} \right>$")
    ax.grid(True, ls='-', lw=0.25, c="gainsboro")
    ax.label_outer()

for idx, ax in enumerate(axs.flat):
    ax.grid(True, ls='-', lw=0.25, c='silver')
    if idx < len(sample):
        gdf = read_snapshot_data(simulation=sample[idx], of="O", to='H')

        mean, bin_edges, _ = binned_statistic(
            x=gdf["CylindricalRadius_ckpc"],
            values=gdf["[O/H]_dex"],
            statistic="mean", bins=N_BINS, range=RXY_RANGE)
        bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2 
        ax.plot(bin_centers, mean, lw=1, color="black", zorder=10)

        # Profile for star-forming gas
        mean, _, _ = binned_statistic(
            x=gdf["CylindricalRadius_ckpc"][gdf["IsStarForming"] == True],
            values=gdf["[O/H]_dex"][gdf["IsStarForming"] == True],
            statistic="mean", bins=N_BINS, range=RXY_RANGE)
        ax.plot(bin_centers[np.isfinite(mean)], mean[np.isfinite(mean)],
                lw=0.75, color="tab:red", zorder=10, label="Star-Forming Gas")

        # Profile for cold gas
        mean, _, _ = binned_statistic(
            x=gdf["CylindricalRadius_ckpc"][gdf["IsCold"] == True],
            values=gdf["[O/H]_dex"][gdf["IsCold"] == True],
            statistic="mean", bins=N_BINS, range=RXY_RANGE)
        ax.plot(bin_centers[np.isfinite(mean)], mean[np.isfinite(mean)],
                lw=0.75, color="tab:blue", zorder=10, label="Cold Gas")

        ax.text(x=0.95, y=0.95, size=6.0,
                s=r"$\texttt{" + sample[idx].upper() + "}$",
                ha="right", va="top", transform=ax.transAxes)
    else:
        ax.axis("off")

axs[0, 1].legend(loc="lower left", framealpha=0.0, fontsize=4.0)

fig.savefig("../images/OH_metallicity_profiles/originals.pdf")
plt.close(fig)

In [51]:
fig, ax = plt.subplots(figsize=(2.5, 2.5))

ax.set_xlim(0, 30)
ax.set_ylim(-0.2, 1.2)
ax.set_xlabel(r"$r_{xy}$ [ckpc]")
ax.set_ylabel(r"$\left< \mathrm{[O/H]} \right>$")

ax.grid(True, ls='-', lw=0.25, c='silver')

for galaxy in sample:
    ax.plot(df["BinCenters_ckpc"], df[f"{galaxy.upper()}_<[O/H]>_dex"],
            lw=1, color="gainsboro", zorder=10)

ax.plot(df["BinCenters_ckpc"], df["SampleAverage_<[O/H]>_dex"],
        lw=1, color='k', zorder=11)
ax.plot(df["BinCenters_ckpc"],
        df["SampleAverage_<[O/H]>_dex"] - df["SampleDispersion_<[O/H]>_dex"],
        lw=1, color='k', zorder=11, ls="--")
ax.plot(df["BinCenters_ckpc"],
        df["SampleAverage_<[O/H]>_dex"] + df["SampleDispersion_<[O/H]>_dex"],
        lw=1, color='k', zorder=11, ls="--")

fig.savefig(
    f"../images/OH_metallicity_profiles/originals_average.pdf")
plt.close(fig)